In [1]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [2]:
lang = 'ru'
lang_entry = 'ru'

In [3]:
# read the csv file
df = pd.read_csv('../generations/meta-llama2_7b_' + lang +  '_2epoch_decoding.csv')
df['data points'] = np.nan
df

,triples,predictions,actuals,data points
0,Peter_Stöger club FC_Admira_Wacker_Mödling,"Peter Stoger играет за ФК ""Адмира Wacker Modli...","Клуб ""Питер Стёгер"" - это ФК ""Адмира Wacker Mö...",NaN
1,Arem-arem ingredient Banana_leaf,Банановая листья - это ингредиент Arem arem.,Банановый лист - это ингредиент в Arem arem.,NaN
2,Weymouth_Sands followedBy Maiden_Castle_(novel...,"Роман ""Майден замок"" последовал за Гластонбери...","Следом за ""Гластонбери Романс"" последовали Wey...",NaN
3,Christian_Panucci club Genoa_C.F.C. Christian_...,Christian Panucci играет за сборную Италии по ...,Christian Panucci играет за Genoa C.C. и сборн...,NaN
4,Chicharrón region Andalusia,ЧИЧАРРОН родом из Андалусии.,Chicharrón найден в районе Андалусия.,NaN
...,...,...,...,...
995,300_North_LaSalle floorCount 60,300 North LaSalle имеет 60 этажей.,300 North LaSalle имеет 60 этажей.,NaN
996,United_States leader Joe_Biden,Джо Байден - лидер США.,Джо Байден - лидер Соединенных Штатов.,NaN
997,United_States leaderTitle President_of_the_Uni...,Лидер США известен как президент США.,Звание лидера США занимает президент США.,NaN
998,Chicago country United_States Chicago isPartOf...,"300 North LaSalle находится в Чикаго, который ...","300 North LaSalle находится в Чикаго, округ Ду...",NaN


In [19]:
sets = ['dev', 'train']
triples = ['1', '2', '3', '4', '5', '6', '7']

path = "../../../../WebNLG/" + lang_entry + "/test"

# Ciclo basato sugli indici del DataFrame
for i in range(len(df)):
    actual = df.at[i, 'actuals']
    triples_set = df.at[i, 'triples']
    data_points = []
    flag = False
    print(f"Entry {i + 1}")
    if lang == 'ru':
        print(f"Triples: {triples_set}")
    else:
        print(f"Actual: {actual}")

    for set in sets:
        for triple in triples:
            path = "../../../../WebNLG/" + lang_entry + "/" + set + "/" + triple + "triples"
    
            for file_name in os.listdir(path):
                if os.path.isfile(os.path.join(path, file_name)):
                    url = os.path.join(path, file_name)
                    tree = ET.parse(url)
                    root = tree.getroot()
                    
                    for entry in root.iter('entry'):

                        if lang == 'ru':                       
                            for modifiedtripleset in entry.iter('modifiedtripleset'):
                                mtriples = ""

                                ts = []
                                for mtriple in modifiedtripleset.iter('mtriple'):  
                                    t = mtriple.text.split(" | ")
                                    t = (t[0], t[1], t[2])
                                    ts.append(t)

                                ts.sort(key=lambda x: x[1])
                                for t in ts:
                                    mtriples += t[0] + " " + t[1] + " " + t[2] + " "


                                #remove all space in mtriples
                                mtriples_ws = mtriples.replace(" ", "")
                                triples_set_ws = triples_set.replace(" ", "")

                                if mtriples_ws == triples_set_ws:
                                    flag = True
                                    for lex in entry.iter('lex'):
                                        if lex.attrib['lang'] == lang:
                                            data_points.append(lex.text)

                                if flag:
                                    break
                                else:
                                    data_points = []
                        else:
                            for lex in entry.iter('lex'):
                                if lang == 'de':
                                    data_points.append(lex.find('text').text)
                                    if lex.find('text').text == actual:
                                        flag = True
                                else:
                                    if lex.attrib['lang'] == lang_entry:
                                        data_points.append(lex.text)
                                        if lex.text == actual:
                                            flag = True
                            if flag:
                                break
                            else:
                                data_points = []

                    if flag:
                        print(f"{data_points}")
                        print('\n')

                        # Aggiungi la lista come singolo elemento nel DataFrame per la riga specifica
                        df.at[i, 'data points'] = data_points
                        break
                if flag:
                    break
            if flag:
                break
df

Entry 1
Triples: Peter_Stöger club FC_Admira_Wacker_Mödling
['Клуб Петера Штёгера - это ФК Адмира Ваккер Мёдлинг.', 'Петер Штёгер является игроком Адмира Ваккер Мёдлинг.', 'Петер Штёгер играет за Адмиру Ваккер Мёдлинг.']


Entry 2
Triples: Arem-arem ingredient Banana_leaf
['Банановый лист – это ингредиент в арем-арем.', 'В арем-арем входит ингредиент – банановый лист.', 'Арем-арем содержит в себе банановый лист.']


Entry 3
Triples: Weymouth_Sands followedBy Maiden_Castle_(novel) A_Glastonbury_Romance followedBy Weymouth_Sands
Entry 4
Triples: Christian_Panucci club Genoa_C.F.C. Christian_Panucci club Italy_national_football_team A.S._Livorno_Calcio manager Christian_Panucci
['ФК "Ливорно" находится под управлением Кристиана Пануччи, который играл в футбол в ФК "Дженоа" и в составе сборной Италии по футболу.', 'Кристиан Пануччи играет за Дженоа и сборную Италии, он также управляет Ливорно.', 'ФК Ливорно управляет Кристиан Пануччи, который играл за сборную Италии по футболу и прикреплен

,triples,predictions,actuals,data points
0,Peter_Stöger club FC_Admira_Wacker_Mödling,"Peter Stoger играет за ФК ""Адмира Wacker Modli...","Клуб ""Питер Стёгер"" - это ФК ""Адмира Wacker Mö...",[Клуб Петера Штёгера - это ФК Адмира Ваккер Мё...
1,Arem-arem ingredient Banana_leaf,Банановая листья - это ингредиент Arem arem.,Банановый лист - это ингредиент в Arem arem.,"[Банановый лист – это ингредиент в арем-арем.,..."
2,Weymouth_Sands followedBy Maiden_Castle_(novel...,"Роман ""Майден замок"" последовал за Гластонбери...","Следом за ""Гластонбери Романс"" последовали Wey...",NaN
3,Christian_Panucci club Genoa_C.F.C. Christian_...,Christian Panucci играет за сборную Италии по ...,Christian Panucci играет за Genoa C.C. и сборн...,"[ФК ""Ливорно"" находится под управлением Кристи..."
4,Chicharrón region Andalusia,ЧИЧАРРОН родом из Андалусии.,Chicharrón найден в районе Андалусия.,"[Чичаррон можно найти в регионе Андалусии., Чи..."
...,...,...,...,...
995,300_North_LaSalle floorCount 60,300 North LaSalle имеет 60 этажей.,300 North LaSalle имеет 60 этажей.,NaN
996,United_States leader Joe_Biden,Джо Байден - лидер США.,Джо Байден - лидер Соединенных Штатов.,"[Джо Байден - лидер США., Лидером США является..."
997,United_States leaderTitle President_of_the_Uni...,Лидер США известен как президент США.,Звание лидера США занимает президент США.,[Лидер Соединенных Штатов известен как президе...
998,Chicago country United_States Chicago isPartOf...,"300 North LaSalle находится в Чикаго, который ...","300 North LaSalle находится в Чикаго, округ Ду...","[Бул. Север Ласаль 300 находится в Чикаго, окр..."


In [21]:
df[df['data points'].isnull()]

,triples,predictions,actuals,data points
2,Weymouth_Sands followedBy Maiden_Castle_(novel...,"Роман ""Майден замок"" последовал за Гластонбери...","Следом за ""Гластонбери Романс"" последовали Wey...",NaN
5,A_Loyal_Character_Dancer country United_States...,"Loyal Charter Dancer опубликован в США, где го...","Loyal Charter Dancer опубликован в США, стране...",NaN
7,A.T._Charlie_Johnson almaMater Harvard_Univers...,"А.Т. Чарли Джонсон, проживающий в США, являетс...",T Charlie Johnson живет в США и является редак...,NaN
18,1634:_The_Bavarian_Crisis author Eric_Flint 16...,"1634: Баварский кризис, написанный Эриком Флин...",1634: Баварский кризис написан на английском я...,NaN
22,A_Glastonbury_Romance author John_Cowper_Powys...,"John Cowper Powys родился в Ширли, Дербышир и ...",Автор A Glastonbury Romance - John Cowper Powy...,NaN
...,...,...,...,...
978,Akita_Museum_of_Art floorArea 3746.66 (square ...,"Музей искусств ""Акита"" был открыт 28 сентября ...","Музей искусств ""Акита"", открытый 28 сентября 2...",NaN
987,"Auburn,_Washington populationDensity 914.8 (in...","Auburn, Washington имеет плотность населения в...","Плотность населения Auburn, Washington, состав...",NaN
988,Asam_pedas country Malaysia Malaysia ethnicGro...,"ASAM PEDAS - это еда, найденная в Малайзии и С...","ASAM PEDAS - это еда, найденная в Малайзии и С...",NaN
994,"1634:_The_Bavarian_Crisis author ""Virginia DeM...","1634: Баварский кризис, написанный Вирджинией ...","Автор Virginia DeMarce и Eric Flint, 1634: ""Ба...",NaN


In [22]:
#save the cols "data_points" into csv file
df['data points'].to_csv(lang + '_data_points.csv', index=False)